In [1]:
import pandas as pd
import os
import pyarrow
import openpyxl
import pickle
from dataclasses import dataclass
import os

print(os.getcwd())


C:\devs\ts\LTC2023_15\exgauster


In [2]:
if 0:
    from exgauster.utils import load_resampled_data

    exgausters_data = load_resampled_data()

    with open('prepared_data.pkl', 'wb') as f:
        pickle.dump(exgausters_data, f, fix_imports=True, buffer_callback=None)


In [3]:
with open('prepared_data.pkl', "rb") as f:
    exgausters_data = pickle.load(f)


C:\tools\miniconda3\envs\stal\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import lgbm_forecaster_custom
from importlib import reload

import matplotlib.pyplot as plt

from typing import Optional
import exgauster.trained_booster

reload(exgauster.utils)
reload(lgbm_forecaster_custom)
reload(exgauster.trained_booster)


m3_hyperparametres = dict(
    n_estimators=5000,  # default 100 - число обрезанных деревьев, alias = num_iterations
    # max_depth=1000,
    num_leaves=255,
    # learning_rate=0.04,
    learning_rate=0.2,
    # boosting='dart', # ?? gbdt быстрее существенно
    min_data_in_leaf=1,
    lambda_l1=0.01,
    lambda_l2=0.05,
    min_sum_hessian_in_leaf=20,  # feature_fraction=0.8,
    # min_sum_hessian_in_leaf=100,  # feature_fraction=0.8,
    # ndcg_eval_at=[1, 3, 5, 10],
)


# Гиперпараметры для случаев "2 пика на всю обучающую выборку"
m1_peak_hyperparams = dict(
    n_estimators=10000,  # default 100 - число обрезанных деревьев, alias = num_iterations
    # max_depth=1000,
    num_leaves=255,
    # learning_rate=0.04,
    learning_rate=0.2,
    # boosting='dart', # ?? gbdt быстрее существенно
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,  # feature_fraction=0.8,
    # ndcg_eval_at=[1, 3, 5, 10],
)

@dataclass
class TrainExgausterConfig:
    hyperparametres: dict
    window_size: int
    booster: Optional[exgauster.trained_booster.TrainedBooster]
    booster_scoring: Optional[exgauster.trained_booster.TrainedBooster]


exgauster_map = {
    exgauster_num: {
        'm1': {
            feature_num: TrainExgausterConfig(
                hyperparametres=m1_peak_hyperparams,
                window_size=40,
                booster=None,
                booster_scoring=None
            )
            for feature_num in range(len(exgausters_data[exgauster_num].y.m1))
        },
        'm3': {
            feature_num: TrainExgausterConfig(
                hyperparametres=m3_hyperparametres,
                window_size=300,
                booster=None,
                booster_scoring=None
            )
            for feature_num in range(len(exgausters_data[exgauster_num].y.m3))
        }
    }
    for exgauster_num in [4, 5, 6, 7, 8, 9]
}
# exgauster_map[4]['m1'][0] = TrainExgausterConfig(hyperparametres=m1_peak_hyperparams, window_size=75, booster=None)

# booster = train_booster(exgauster_num=4, feature_num=0, is_m1=True)

# with open('trained_boosters_with_test_score.pkl', 'rb') as f:
#     exgauster_map = pickle.load(f)






In [5]:
print(len(exgausters_data[4].y.m3))

23


In [ ]:

def train_booster(exgauster_num: int, feature_num: int, is_m1: bool, train_test_split_factor: Optional[float] = 0.8):
    cfg: TrainExgausterConfig = exgauster_map[exgauster_num]['m1' if is_m1 else 'm3'][feature_num]

    return exgauster.trained_booster.train_booster(
        exgausters_data[exgauster_num].train,
        is_m1=is_m1,
        target_labels=exgausters_data[exgauster_num].y.m1[feature_num] if is_m1 else exgausters_data[exgauster_num].y.m3[feature_num],
        train_test_split_factor=train_test_split_factor,
        hyperparams=cfg.hyperparametres,
        window_size=cfg.window_size,
    )


for exgauster_num in [8, 9]:
    for feature_num in range(len(exgausters_data[exgauster_num].y.m1)):
        if exgauster_num == 8 and feature_num < 6:
            continue
        for (m, is_m1) in [('m1', True), ('m3', False)]:
            cfg: TrainExgausterConfig = exgauster_map[exgauster_num][m][feature_num]
            if cfg.booster is not None:
                continue

            print(f'Finale training exgauster {exgauster_num} {m} {feature_num}...')
            cfg.booster = train_booster(
                exgauster_num=exgauster_num,
                feature_num=feature_num,
                is_m1=is_m1,
                train_test_split_factor=None
            )
            exgauster.trained_booster.plot_train_results(
                cfg.booster,
                exgauster_num=f'final {exgauster_num}',
                feature_num=feature_num,
                save_to_file=True
            )
            with open('final_trained_boosters_with_test_score8-9.pkl', 'wb') as f:
                pickle.dump(exgauster_map, f, fix_imports=True, buffer_callback=None)


for exgauster_num in []:
    for feature_num in range(len(exgausters_data[exgauster_num].y.m1)):
        for (m, is_m1) in [('m1', True), ('m3', False)]:
            cfg: TrainExgausterConfig = exgauster_map[exgauster_num][m][feature_num]
            if cfg.booster_scoring is not None:
                continue

            print(f'training exgauster {exgauster_num} {m} {feature_num}...')
            cfg.booster_scoring = train_booster(
                exgauster_num=exgauster_num,
                feature_num=feature_num,
                is_m1=is_m1,
                train_test_split_factor=0.8
            )
            exgauster.trained_booster.plot_train_results(
                cfg.booster_scoring,
                exgauster_num=exgauster_num,
                feature_num=feature_num,
                save_to_file=True
            )
            with open('trained_boosters_with_test_score.pkl', 'wb') as f:
                pickle.dump(exgauster_map, f, fix_imports=True, buffer_callback=None)

Finale training exgauster 8 m1 6...
All of Y_ЭКСГАУСТЕР А/М №8_ЭЛ/ДВИГАТЕЛЬ ГАЗ. ЗАДВИЖКИ ЭКСГ. №8 M1 is 0, skipping learn
Finale training exgauster 8 m3 6...
Training LGBMForecasterCustom...


Inferred granularity <5 * Minutes>


train time: 3.3058287616666346 min
all booster dataset is train dataset, nothing to plotting
Finale training exgauster 8 m1 7...
All of Y_ЭКСГАУСТЕР А/М №8_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №8 M1 is 0, skipping learn
Finale training exgauster 8 m3 7...
Training LGBMForecasterCustom...


Inferred granularity <5 * Minutes>


train time: 3.570420015000127 min
all booster dataset is train dataset, nothing to plotting
Finale training exgauster 8 m1 8...
Training LGBMForecasterCustom...


...

Inferred granularity <5 * Minutes>


train time: 4.029464226666702 min
all booster dataset is train dataset, nothing to plotting
Finale training exgauster 8 m1 22...
All of Y_ЭКСГАУСТЕР А/М №8_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №8 M1 is 0, skipping learn
Finale training exgauster 8 m3 22...
Training LGBMForecasterCustom...


Inferred granularity <5 * Minutes>


train time: 4.332822483333439 min
all booster dataset is train dataset, nothing to plotting
Finale training exgauster 8 m1 23...
All of Y_ЭКСГАУСТЕР А/М №8_ЗАДВИЖКА ЭКСГ. №8 M1 is 0, skipping learn
Finale training exgauster 8 m3 23...
Training LGBMForecasterCustom...


Inferred granularity <5 * Minutes>
